In [2]:
import os
import json
filename_list = os.listdir('../data')
filename_list = sorted(filename_list, key=lambda x: (len(x), x))
tag_2 = filename_list[0]
filename_list = filename_list[1:]
print(tag_2, filename_list, len(filename_list))
rel_list = []
with open('../data/' + tag_2, 'r') as f:
    lines = f.readlines()
    lines = list(map(lambda x: x.replace('\n', ''), lines))
    f_list = list(map(lambda x: x.split(' ')[0], lines))
    rel_list = list(map(lambda x: x.split(' ')[1], lines))
    temp = sorted(zip(f_list, rel_list), key=lambda x: (len(x[0]), x[0]))
    f_list, rel_list = zip(*temp)
    f_list = list(f_list)
    rel_list = list(rel_list)
    print(f_list)
    print(rel_list)

tag_2.txt ['1_data1.txt', '2_data1.txt', '3_data1.txt', '4_data1.txt', '5_data1.txt', '6_data1.txt', '7_data1.txt', '8_data1.txt', '9_data1.txt', '10_data1.txt', '11_data1.txt', '12_data1.txt', '13_data1.txt', '14_data1.txt', '15_data1.txt', '16_data1.txt', '17_data1.txt', '18_data1.txt', '19_data1.txt', '20_data1.txt', '21_data1.txt', '22_data1.txt', '23_data1.txt', '24_data1.txt', '25_data1.txt', '26_data1.txt', '27_data1.txt', '28_data1.txt', '29_data1.txt', '30_data1.txt', '32_data1.txt', '34_data1.txt', '35_data1.txt', '36_data1.txt', '37_data1.txt', '38_data1.txt', '39_data1.txt', '40_data1.txt', '41_data1.txt', '42_data1.txt', '43_data1.txt', '44_data1.txt', '45_data1.txt', '46_data1.txt', '47_data1.txt', '48_data1.txt', '49_data1.txt', '50_data1.txt', '51_data1.txt', '52_data1.txt', '53_data1.txt', '54_data1.txt', '55_data1.txt', '56_data1.txt', '57_data1.txt', '58_data1.txt', '59_data1.txt', '60_data1.txt', '61_data1.txt', '62_data1.txt', '63_data1.txt', '64_data1.txt', '65_da

In [3]:
dataset_list = []
for filename, rel in zip(filename_list, rel_list):
    dataset = {}
    with open('../data/' + filename, 'r') as f:
        lines = f.readlines()
        filename = filename.replace('1.txt', '.txt')
        dataset['filename'] = filename
        lines = list(map(lambda x: x.replace('\n', ''), lines))
        data = list(map(lambda x: x.split(' ')[0], lines))
        label = list(map(lambda x: x.split(' ')[1], lines))
        text = ' '.join(data)
        dataset['token'] = data
        dataset['bio_label'] = label
        dataset['text'] = text
        dataset['relation'] = rel
        dataset_list.append(dataset)

In [4]:
dataset_list[0]['relation']

'110000300001100311010'

In [5]:
# 处理存在相邻B-signal、B-condition的情况
# 如：B-signal B-signal -> B-signal B-signal
for dataset in dataset_list:
    bio_label = dataset['bio_label']
    modified_bio_label = []
    pre_token = None
    for token in bio_label:
        if token == 'B-signal' and pre_token == 'B-signal':
            modified_bio_label.append('I-signal')
        elif token == 'B-condition' and pre_token == 'B-condition':
            modified_bio_label.append('I-condition')
        else:
            modified_bio_label.append(token)
        pre_token = token
    dataset['bio_label'] = modified_bio_label


In [6]:
# 把B-activity、I-activity合并为[activity]
# 把B-condition、I-condition合并为[condition]
# B-signal、I-signal保留原文中的信号词
for dataset in dataset_list:
    bio_label = dataset['bio_label']
    token = dataset['token']
    data_2_mask = []
    for word, bio in zip(token, bio_label):
        if bio == 'B-activity':
            data_2_mask.append('[activity]')
        elif bio in ['B-signal', 'I-signal']:
            data_2_mask.append(word)
        elif bio == 'B-condition':
            data_2_mask.append('[condition]')
        else:
            pass
    dataset['data_2_mask'] = data_2_mask

In [7]:
# 按照信号词在文本中的顺序提取出信号词或信号词组
# [['If'], ['If'], ['After'], ['if'], ['but', 'if'], ['After'], ['in', 'both', 'cases']]
for dataset in dataset_list:
    data_2_mask = dataset['data_2_mask']
    signal_token_list = []
    current_token_list = []
    for token in data_2_mask:
        if token not in ['[activity]', '[condition]']:
            current_token_list.append(token)
        elif current_token_list:
            signal_token_list.append(current_token_list)
            current_token_list = []
    if current_token_list:
        signal_token_list.append(current_token_list)
    dataset['signal_token_list'] = signal_token_list

In [8]:
# 将原有的的data_2_mask序列中的相邻信号词合并，并按先后顺序标号：signal-{count}
for dataset in dataset_list:
    data_2_mask = dataset['data_2_mask']
    data_2_mask_single_signal = []
    current_token_list = []
    signal_count = 0
    for token in data_2_mask:
        if token not in ['[activity]', '[condition]']:
            current_token_list.append(token)
        elif current_token_list:
            signal_count += 1 
            data_2_mask_single_signal.append(f'signal-{signal_count}')
            current_token_list = []
            data_2_mask_single_signal.append(token)  
        else:
            data_2_mask_single_signal.append(token)
    dataset['data_2_mask_single_signal'] = data_2_mask_single_signal

In [9]:
# 使用LLM对信号词分类
import requests
url = 'http://localhost:11434/api/generate'
for dataset in dataset_list:
    signal_token_list = dataset['signal_token_list']
    signal_token_llm_list = []
    for signal_token in signal_token_list:
        words = ' '.join(signal_token)
        data = {
            'model': 'llama3:70b-instruct',
            'prompt': f'''
            I want to extract activities, and determine the relationship between two activities from a business process text described in natural language. Below I will give you a word or phrase, denoted by [words], which connects two activities, i.e. activity1-[words]-activity2, please judge the type of relationship between two activities based on this words, including 4 types of relationship, as follows:
            [sign-selection]: indicates a selection relationship, the most common is to select an activity to be executed under certain conditions.
            [sign-successor]: indicates the direct sequential relationship between activities
            [sign-parallel]: indicates a concurrent relationship between activities.
            [sign-loop]: indicates a circular relationship between activities.
            The [words] I gave you are "[{words}]", giving the type of relationship(Just answer the type and do not output any extra content).
            ''',
            'stream': False,
        }
        try: 
            response = requests.post(url, json=data)
            result = response.json()['response']
            signal_token_llm_list.append(result)
        except:
            dataset_list_json = json.dumps(dataset_list)
            with open('datasets2.json', 'w') as f:
                f.write(dataset_list_json)
    dataset['signal_token_llm_list'] = signal_token_llm_list
    print(dataset['filename'], signal_token_llm_list)
dataset_list_json = json.dumps(dataset_list)
with open('datasets2.json', 'w') as f:
    f.write(dataset_list_json)

1_data.txt ['[sign-selection]', '[sign-selection]', '[sign-successor]', '[sign-selection]', '[sign-selection]', '[sign-successor]', '[sign-selection]']
2_data.txt ['[sign-selection]', '[sign-selection]', '[sign-selection]', '[sign-selection]', '[sign-selection]', '[sign-parallel]', '[sign-selection]']
3_data.txt ['[sign-successor]', '[sign-successor]', '[sign-selection]', '[sign-selection]', '[sign-selection]', '[sign-successor]', '[sign-selection]', '[sign-selection]', '[sign-successor]', '[sign-successor]', '[sign-successor]', '[sign-successor]', '[sign-successor]', '[sign-selection]']
4_data.txt ['[sign-parallel]', '[sign-successor]', '[sign-parallel]']
5_data.txt ['[sign-successor]', '[sign-successor]', '[sign-selection]', '[sign-selection]', '[sign-successor]', '[sign-successor]', '[sign-successor]', '[sign-selection]', '[sign-loop]', '[sign-selection]', '[sign-successor]', '[sign-successor]', '[sign-successor]', '[sign-selection]', '[sign-successor]']
6_data.txt ['[sign-successor

In [4]:
# API测试
import requests
url = 'http://localhost:11434/api/generate'
data = {
    'model': 'llama3:70b-instruct',
    'prompt':  '''
            I want to extract activities, and determine the relationship between two activities from a business process text described in natural language. Below I will give you a word or phrase, denoted by [words], which connects two activities, i.e. activity1-[words]-activity2, please judge the type of relationship between two activities based on this words, including 4 types of relationship, as follows:
            [sign-selection]: indicates a selection relationship, the most common is to select an activity to be executed under certain conditions.
            [sign-successor]: indicates the direct sequential relationship between activities
            [sign-parallel]: indicates a concurrent relationship between activities.
            [sign-loop]: indicates a circular relationship between activities.
            The [words] I gave you are "in both cases", giving the type of relationship(Just answer the type and do not output any extra content).
        ''',
    'stream': False,
}

response = requests.post(url, json=data)
print(response.json()['response'])

[sign-selection]


In [18]:
import json
dataset_list = None
with open ('datasets3.json', 'r') as f:
    dataset_list = json.load(f)
for dataset in dataset_list:
    
    token = dataset['token']
    bio_label = dataset['bio_label']
    data_2_mask_single_signal_llm = dataset['data_2_mask_single_signal_llm']
    node_token_list = []
    current_node_token = []
    current_node_type = None
    for word, bio in zip(token, bio_label):
        # print(word, bio)
        if bio == 'punctuation' or bio =='O':
            if current_node_token:
                node_token_list.append(' '.join(current_node_token))
            current_node_token = []
            current_node_type = None
            continue  # 忽略 punctuation
        is_beginning_tag= bio.startswith('B-')
        current_node_key = bio.split('-')[1] if not is_beginning_tag or bio != 'punctuation' or bio != 'O' else None  # 获取组的类型，如 'activity', 'condition', 'signal'
        # 如果当前组是空或当前项的类型与之前不同，则需要开始一个新组
        if current_node_type is None or current_node_key != current_node_type:
            if current_node_token:  # 如果当前组不为空，进行拼接
                node_token_list.append(' '.join(current_node_token))
            current_node_token = [word]  # 开始新的组
            current_node_type = current_node_key
        else:
            current_node_token.append(word)  # 添加到当前组

    # 最后处理遗留的组
    if current_node_token:
        node_token_list.append(' '.join(current_node_token))
    if len(node_token_list) != len(data_2_mask_single_signal_llm):
        for word, bio in zip(token, bio_label):
            print(word, bio)
        print(dataset['filename'])
        print(len(node_token_list) , len(data_2_mask_single_signal_llm))
        print(node_token_list)
    # dataset['node_token_list'] = node_token_list

3_data.txt
39 40
['As soon as', 'an employee files an expense report the process is started', 'Once', 'the expense report is received a check is performed to make sure the employee has an account', 'If', 'this check does not take place within 28 days', "the process is stopped and the employee is sent a ' resubmit ' email", 'When', 'the employee has an account', 'the report is reviewed', 'If', 'the employee does not have an account', 'an account is created', 'after', 'which', 'the report is reviewed', 'A check is performed to see if the amount of the expense exceeds 200', 'If', 'it does not exceed 200', "the employee is reimbursed through a deposit directly to the employee 's bank account and the process is ended", 'In the case that', 'it does exceed 200', 'a supervisor is asked for approval', 'Three things can happen after this', 'First', 'the supervisor approves within 7 days', 'this will result in', "the deposit of the reimbursement directly to the employee 's bank account", 'Once', 

In [41]:
def split_list(bio_label, token):
    result = []  # 存储最终结果
    stack = []  # 使用列表作为栈来存储子列表
    for word, bio in zip(token, bio_label):
        if bio.startswith('B-'):
            if stack:
                result.append(stack)
                stack = []
            stack.append(word)  # 将当前元素添加到新的子列表中
        elif bio.startswith('I-'):
            if stack:
                stack.append(word)  # 将当前元素添加到栈顶的子列表中
        else:
            pass  # 这里暂时不处理，可以根据需求添加代码
    if stack:
        result.append(stack)
    return result
# 测试代码
bio_label = ['B-activity', 'I-activity', 'I-activity', 'B-activity', 'I-activity']
token = ['a', 'b', 'c', 'd', 'e']
b = split_list(bio_label, token)
print(b)

[['a', 'b', 'c'], ['d', 'e']]


In [43]:
import json
dataset_list = None
with open ('datasets3.json', 'r') as f:
    dataset_list = json.load(f)
for dataset in dataset_list:
    token = dataset['token']
    bio_label = dataset['bio_label']
    signal_token_list = dataset['signal_token_list']
    data_2_mask_single_signal_llm = dataset['data_2_mask_single_signal_llm']
    new_token = []
    new_bio_label = []
    for word, bio in zip(token, bio_label):
        if bio == 'punctuation' or bio == 'O' or bio == 'B-signal' or bio == 'I-signal':
            continue
        else:
            new_token.append(word)
            new_bio_label.append(bio)
    result = split_list(new_bio_label, new_token)
    # print(len(result) + len(dataset['signal_token_list']) == len(dataset['data_2_mask_single_signal_llm']))
    i = 0
    j = 0
    node_token_list = []
    for item in data_2_mask_single_signal_llm:
        if item in ['[activity]', '[condition]']:
            node_token_list.append(' '.join(result[i]))
            i += 1
        else:
            node_token_list.append(' '.join(signal_token_list[j]))
    print(node_token_list)
    dataset['node_token_list'] = node_token_list
with open('datasets4.json', 'w') as f:
    json.dump(dataset_list, f)

['The credit company receives the credit information from the customer', 'If', 'the requested amount is greater than 1M $', 'an approval must be requested', 'If', 'the requested amount is lower or equal to 1M $', 'the company assess the risk of the credit', 'If', 'the assessment', 'If', 'the risk is high', 'an approval must be requested', 'If', 'the risk is low', 'the credit is accepted', 'If', 'the approval request', 'the credit could be accepted or rejected', 'If', 'an email is sent to the customer']
['If', 'a not collected bag is identified', 'CVS must check the number of resolution retries conducted', 'If', 'the number of retries is greater than an specified number', 'the bag must be returned to the production area for disassemble', 'If', 'CVS must try to contact the customer by phone and remind to pick up the bag', 'If', 'the contact is successful and the customer withdrawn the order', 'the bag must be returned to the production area for disassemble', 'If', 'it is successful and t

In [11]:
dataset_list_json = json.dumps(dataset_list)
with open('datasets3.json', 'w') as f:
    f.write(dataset_list_json)

In [2]:
text = "Hello, WORLD![]]"
lowercase_text = text.casefold()

print(lowercase_text)
print(text)

hello, world![]]
Hello, WORLD![]]
